In [5]:
import pandas as pd
import numpy as np
import warnings
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras import callbacks
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from keras.layers import Dense, Dropout
from sklearn.model_selection import KFold
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

warnings.filterwarnings(action='ignore')
tf.config.set_visible_devices([], 'GPU')

In [6]:
df = pd.read_csv("data/modified_0420.csv")
x_train = df.drop(columns=['loan_status','Unnamed: 0'])
y_train = df['loan_status']

In [7]:
nan_indices = np.isnan(x_train).any(axis=1)
x_train = x_train[~nan_indices]
y_train = y_train[~nan_indices]

In [8]:
# minmax scaler 찾아보기
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x_train)

# PCA로 차원 축소
pca = PCA(n_components=50)
X_pca = pca.fit_transform(X_scaled)
x_train = X_pca.astype('float32')
# stratify로 학습에 용이하게 비율 유지
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
y_train=to_categorical(y_train, 2).astype(int)
y_validation=to_categorical(y_validation, 2).astype(int)

In [11]:
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())    
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

### without_earlystopping

In [15]:
model_file_path = "dnn_models/log_transformed/pca95_feature50/without_earlystopping_0421.h5"
valid_accs, valid_f1s, valid_recalls, valid_precisions = [], [], [], []

# GPU 설정
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)
# KFold validation 사용
# k: n_splits=5
kf = KFold(random_state=30,
           n_splits=5,
           shuffle=True
          )
model = tf.keras.models.load_model(model_file_path, custom_objects={"f1_score":f1_score, "recall":recall, "precision":precision})
for train_index, val_index in kf.split(x_train, y_train):
    X_train_fold, X_val_fold = x_train[train_index], x_train[val_index]
    Y_train_fold, Y_val_fold = y_train[train_index], y_train[val_index]

    # 모델 validation
    valid_loss, valid_acc, valid_f1, valid_recall, valid_precision = model.evaluate(X_val_fold, Y_val_fold)
    valid_accs.append(valid_acc)
    valid_f1s.append(valid_f1)
    valid_recalls.append(valid_recall)
    valid_precisions.append(valid_precision)
    print("===================================")
    print("Validation accuracy:", valid_acc)
    print("Validation F1-score:", valid_f1)
    print("Validation recall:", valid_recall)
    print("Validation precision:", valid_precision)
print("###################################")
print("avg Validation accuracy:", np.mean(valid_accs))
print("avg Validation F1-score:", np.mean(valid_f1s))
print("avg Validation recall:", np.mean(valid_recalls))
print("avg Validation precision:", np.mean(valid_precisions))

5659/5659 [==============================] - 5s 772us/step - loss: 0.4500 - accuracy: 0.8023 - f1_score: 0.8022 - recall: 0.8022 - precision: 0.8022
Validation accuracy: 0.8022632002830505
Validation F1-score: 0.80223548412323
Validation recall: 0.8022355437278748
Validation precision: 0.8022355437278748
5659/5659 [==============================] - 4s 740us/step - loss: 0.4524 - accuracy: 0.8012 - f1_score: 0.8012 - recall: 0.8012 - precision: 0.8012
Validation accuracy: 0.8011862635612488
Validation F1-score: 0.8011748194694519
Validation recall: 0.8011748790740967
Validation precision: 0.8011748790740967
5659/5659 [==============================] - 5s 814us/step - loss: 0.4526 - accuracy: 0.8006 - f1_score: 0.8005 - recall: 0.8005 - precision: 0.8005
Validation accuracy: 0.800551176071167
Validation F1-score: 0.8005477786064148
Validation recall: 0.8005478382110596
Validation precision: 0.8005478382110596
5659/5659 [==============================] - 5s 824us/step - loss: 0.4489 - acc

In [13]:
model_file_path = "dnn_models/log_transformed/pca95_feature50/with_earlystopping_001_0421.h5"
valid_accs, valid_f1s, valid_recalls, valid_precisions = [], [], [], []

# GPU 설정
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)
# KFold validation 사용
# k: n_splits=5
kf = KFold(random_state=30,
           n_splits=5,
           shuffle=True
          )
model = tf.keras.models.load_model(model_file_path, custom_objects={"f1_score":f1_score, "recall":recall, "precision":precision})
for train_index, val_index in kf.split(x_train, y_train):
    X_train_fold, X_val_fold = x_train[train_index], x_train[val_index]
    Y_train_fold, Y_val_fold = y_train[train_index], y_train[val_index]

    # 모델 validation
    valid_loss, valid_acc, valid_f1, valid_recall, valid_precision = model.evaluate(X_val_fold, Y_val_fold)
    valid_accs.append(valid_acc)
    valid_f1s.append(valid_f1)
    valid_recalls.append(valid_recall)
    valid_precisions.append(valid_precision)
    print("===================================")
    print("Validation accuracy:", valid_acc)
    print("Validation F1-score:", valid_f1)
    print("Validation recall:", valid_recall)
    print("Validation precision:", valid_precision)
print("###################################")
print("avg Validation accuracy:", np.mean(valid_accs))
print("avg Validation F1-score:", np.mean(valid_f1s))
print("avg Validation recall:", np.mean(valid_recalls))
print("avg Validation precision:", np.mean(valid_precisions))

5659/5659 [==============================] - 5s 769us/step - loss: 0.4534 - accuracy: 0.8002 - f1_score: 0.8002 - recall: 0.8002 - precision: 0.8002
Validation accuracy: 0.800203263759613
Validation F1-score: 0.8001757264137268
Validation recall: 0.8001757860183716
Validation precision: 0.8001757860183716
5659/5659 [==============================] - 4s 778us/step - loss: 0.4555 - accuracy: 0.7989 - f1_score: 0.7989 - recall: 0.7989 - precision: 0.7989
Validation accuracy: 0.7989164590835571
Validation F1-score: 0.7989051938056946
Validation recall: 0.7989052534103394
Validation precision: 0.7989052534103394
5659/5659 [==============================] - 5s 796us/step - loss: 0.4557 - accuracy: 0.7985 - f1_score: 0.7984 - recall: 0.7984 - precision: 0.7984
Validation accuracy: 0.7984524965286255
Validation F1-score: 0.7984493374824524
Validation recall: 0.7984493970870972
Validation precision: 0.7984493970870972
5659/5659 [==============================] - 4s 784us/step - loss: 0.4521 - a

In [14]:
model_file_path = "dnn_models/log_transformed/pca95_feature50/with_earlystopping_0001_0421.h5"
valid_accs, valid_f1s, valid_recalls, valid_precisions = [], [], [], []

# GPU 설정
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)
# KFold validation 사용
# k: n_splits=5
kf = KFold(random_state=30,
           n_splits=5,
           shuffle=True
          )
model = tf.keras.models.load_model(model_file_path, custom_objects={"f1_score":f1_score, "recall":recall, "precision":precision})
for train_index, val_index in kf.split(x_train, y_train):
    X_train_fold, X_val_fold = x_train[train_index], x_train[val_index]
    Y_train_fold, Y_val_fold = y_train[train_index], y_train[val_index]

    # 모델 validation
    valid_loss, valid_acc, valid_f1, valid_recall, valid_precision = model.evaluate(X_val_fold, Y_val_fold)
    valid_accs.append(valid_acc)
    valid_f1s.append(valid_f1)
    valid_recalls.append(valid_recall)
    valid_precisions.append(valid_precision)
    print("===================================")
    print("Validation accuracy:", valid_acc)
    print("Validation F1-score:", valid_f1)
    print("Validation recall:", valid_recall)
    print("Validation precision:", valid_precision)
print("###################################")
print("avg Validation accuracy:", np.mean(valid_accs))
print("avg Validation F1-score:", np.mean(valid_f1s))
print("avg Validation recall:", np.mean(valid_recalls))
print("avg Validation precision:", np.mean(valid_precisions))

5659/5659 [==============================] - 5s 828us/step - loss: 0.4523 - accuracy: 0.8014 - f1_score: 0.8014 - recall: 0.8014 - precision: 0.8014
Validation accuracy: 0.8013851046562195
Validation F1-score: 0.8013654947280884
Validation recall: 0.8013655543327332
Validation precision: 0.8013655543327332
5659/5659 [==============================] - 5s 818us/step - loss: 0.4542 - accuracy: 0.8012 - f1_score: 0.8012 - recall: 0.8012 - precision: 0.8012
Validation accuracy: 0.8011642098426819
Validation F1-score: 0.8011527061462402
Validation recall: 0.801152765750885
Validation precision: 0.801152765750885
5659/5659 [==============================] - 5s 874us/step - loss: 0.4546 - accuracy: 0.8003 - f1_score: 0.8003 - recall: 0.8003 - precision: 0.8003
Validation accuracy: 0.800346851348877
Validation F1-score: 0.8003434538841248
Validation recall: 0.8003435134887695
Validation precision: 0.8003435134887695
5659/5659 [==============================] - 5s 820us/step - loss: 0.4511 - acc